In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import os

In [2]:
# Paths
base_dir = "../datasets"
train_dir = os.path.join(base_dir, "train")
valid_dir = os.path.join(base_dir, "valid")
test_dir  = os.path.join(base_dir, "test")

In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)
valid_data = valid_test_datagen.flow_from_directory(
    valid_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)
test_data = valid_test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)

Found 286 images belonging to 27 classes.
Found 122 images belonging to 27 classes.
Found 122 images belonging to 27 classes.


In [4]:
# Load pretrained base
base_model = MobileNetV2(input_shape=(*IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze all layers

9406464/9406464 [==============================] - 1s 0us/step


In [6]:
# Build model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_data.num_classes, activation='softmax')
])

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Train
EPOCHS = 20
history = model.fit(train_data, epochs=EPOCHS, validation_data=valid_data)

Epoch 1/20
18/18 [==============================] - 16s 883ms/step - loss: 0.5064 - accuracy: 0.8427 - val_loss: 1.1266 - val_accuracy: 0.6967
Epoch 2/20
18/18 [==============================] - 18s 986ms/step - loss: 0.5201 - accuracy: 0.8531 - val_loss: 1.0328 - val_accuracy: 0.7213
Epoch 3/20
18/18 [==============================] - 18s 1s/step - loss: 0.4245 - accuracy: 0.8951 - val_loss: 1.0848 - val_accuracy: 0.7131
Epoch 4/20
18/18 [==============================] - 15s 843ms/step - loss: 0.4443 - accuracy: 0.8846 - val_loss: 1.0592 - val_accuracy: 0.7623
Epoch 5/20
18/18 [==============================] - 15s 846ms/step - loss: 0.4220 - accuracy: 0.9021 - val_loss: 1.0239 - val_accuracy: 0.6967
Epoch 6/20
18/18 [==============================] - 18s 1s/step - loss: 0.4408 - accuracy: 0.9021 - val_loss: 1.0321 - val_accuracy: 0.7049
Epoch 7/20
18/18 [==============================] - 17s 971ms/step - loss: 0.3877 - accuracy: 0.8951 - val_loss: 1.0939 - val_accuracy: 0.7049
Epoch

In [10]:
# Evaluate
loss, acc = model.evaluate(test_data)
print(f"Test Accuracy: {acc * 100:.2f}%")

8/8 [==============================] - 4s 520ms/step - loss: 0.8981 - accuracy: 0.7459
Test Accuracy: 74.59%


In [11]:

# Save
model.save("mudra_transfer_model74.h5")

In [12]:
# Fine-tune the model
# Unfreeze last few layers
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Freeze all except last 30 layers
    layer.trainable = False

# Recompile with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune
fine_tune_epochs = 10
history_finetune = model.fit(train_data,
                             epochs=fine_tune_epochs,
                             validation_data=valid_data)


Epoch 1/10
18/18 [==============================] - 19s 896ms/step - loss: 2.1161 - accuracy: 0.4021 - val_loss: 1.0294 - val_accuracy: 0.7131
Epoch 2/10
18/18 [==============================] - 16s 867ms/step - loss: 1.6861 - accuracy: 0.5105 - val_loss: 1.0253 - val_accuracy: 0.7131
Epoch 3/10
18/18 [==============================] - 15s 836ms/step - loss: 1.4727 - accuracy: 0.5490 - val_loss: 1.0262 - val_accuracy: 0.7049
Epoch 4/10
18/18 [==============================] - 16s 901ms/step - loss: 1.3400 - accuracy: 0.5734 - val_loss: 1.0217 - val_accuracy: 0.7049
Epoch 5/10
18/18 [==============================] - 18s 995ms/step - loss: 1.1997 - accuracy: 0.6503 - val_loss: 1.0134 - val_accuracy: 0.7131
Epoch 6/10
18/18 [==============================] - 17s 971ms/step - loss: 1.0953 - accuracy: 0.6713 - val_loss: 1.0061 - val_accuracy: 0.7213
Epoch 7/10
18/18 [==============================] - 15s 855ms/step - loss: 1.0103 - accuracy: 0.6923 - val_loss: 1.0066 - val_accuracy: 0.7213

In [ ]:
# Evaluate per-class accuracy
import numpy as np
from sklearn.metrics import classification_report

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)

print(classification_report(test_data.classes, y_pred, target_names=list(test_data.class_indices.keys())))



8/8 [==============================] - 5s 613ms/step
                  precision    recall  f1-score   support

     10_Kabitham       1.00      1.00      1.00         6
  11_Kadagamugam       0.71      0.83      0.77         6
      12_Kangula       0.80      0.80      0.80         5
 13_Katharimugam       0.00      0.00      0.00         2
      14_Mayuram       0.50      0.75      0.60         4
 15_Mirgachirsha       0.00      0.00      0.00         2
      16_Mukulam       1.00      1.00      1.00         1
       17_Mushti       1.00      0.43      0.60         7
   18_Padmakosha       1.00      0.75      0.86         4
     19_Pathakam       0.50      1.00      0.67         5
     1_Alapadmam       1.00      0.50      0.67         4
  20_Sarpashisha       1.00      0.88      0.93         8
     21_Shikaram       0.60      1.00      0.75         6
 22_Shukathundam       0.88      0.70      0.78        10
   23_Simhamugha       0.00      0.00      0.00         1
        24_Suchi  

c:\Users\amrit\.conda\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\amrit\.conda\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\amrit\.conda\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Save class labels
import json

label_map = train_data.class_indices
with open("mudra_labels.json", "w") as f:
    json.dump(label_map, f)
